In [1]:
import os
%pwd

'f:\\ml_projects\\text_summarizer\\notbooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml_projects\\text_summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    data_validation_status_file: str
    data_validation_required_file: list
    unzip_dir:Path

In [4]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories

class ConfigurationManager():
    def __init__(self,config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        temp_config = self.config.data_validation
        create_directories([temp_config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir= Path(temp_config.root_dir),
            data_validation_status_file = temp_config.data_validation_status_file,
            data_validation_required_file = temp_config.data_validation_required_file,
            unzip_dir = temp_config.unzip_dir
        )

        return data_validation_config

In [5]:
import os,sys
from text_summarizer.utils.logger import logger
from text_summarizer.utils.exception import CustomException

In [8]:
class DataValidation:
    def __init__(self,config:DataValidationConfig) -> None:
        self.config = config
    
    def is_valid(self):
        validation_status = None
        all_required_files = self.config.data_validation_required_file
        file_that_exists = os.listdir(self.config.unzip_dir)
        try:
            for file in all_required_files:
                if file not in file_that_exists:
                    validation_status = False
                    with open(self.config.data_validation_status_file,'w') as file_obj:
                        file_obj.write(f'Validation Status = {validation_status}')
                        logger.info(f"Validation Status is {validation_status}")

                        raise SystemExit
                else:
                    validation_status = True
                    with open(self.config.data_validation_status_file,'w') as file_obj:
                        file_obj.write(f'Validation Status = {validation_status}')
            logger.info(f"Validation Status is {validation_status}")
        except Exception as e:
            raise CustomException(e,sys)

In [9]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.is_valid()
except Exception as e:
    raise CustomException(e,sys)

 [ 2024-01-09 11:50:55,261 ] - 25 - common - artifacts Directory Created Successfully
 [ 2024-01-09 11:50:55,264 ] - 25 - common - artifacts/data_validation Directory Created Successfully
['validation', 'train', 'test', 'dataset_dict.json']
['dataset_dict.json', 'test', 'train', 'validation']
 [ 2024-01-09 11:50:55,270 ] - 24 - 420452909 - Validation Status is True
